In [64]:
import numpy as np
from scipy.linalg import lu
from scipy.interpolate import lagrange
from welchberlekamp import makeEncoderDecoder

# F_2
http://www.math.unl.edu/~s-jeverso2/McElieceProject.pdf <br>
$\mathbb{F}_{2}$

# F_23
https://www.youtube.com/watch?v=GNgOJN9LD-I <br>
https://jeremykun.com/2015/09/07/welch-berlekamp/ <br>
$\mathbb{F}_{23}$

In [50]:
n = 23

In [51]:
m = np.array([1, 2, 3], dtype = int)

In [52]:
N = 6
K = 3
T = 1

In [53]:
G = np.array([
    [1, 1, 1, 1, 1, 1],
    [0, 1, 2, 3, 4, 5],
    [0, 1, 4, 9, 16, 25]], dtype = int) % n

P = np.array([
    [1, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0],
    [0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 1],
    [0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 1, 0]], dtype = int) % n

S = np.array([
    [1, 1, 3],
    [4, 1, 6],
    [7, 2, 9]], dtype = int) %n

In [54]:
SGP = S.dot(G).dot(P) % n
SGP

array([[ 1, 15,  7,  5, 12,  8],
       [ 4,  7, 12, 11, 21, 15],
       [ 7,  1, 21, 18, 12,  2]])

In [55]:
c = m.dot(SGP) % n
c

array([ 7,  9,  2, 12, 21, 21])

In [56]:
ei = np.random.randint(c.shape[0])
ev = np.random.randint(n)
err = np.zeros(c.shape[0], dtype = int)
err[ei] = ev
err = np.array([0, 6, 0, 0, 0, 0])
err

array([0, 6, 0, 0, 0, 0])

In [57]:
c = (c + err) % n
c

array([ 7, 15,  2, 12, 21, 21])

In [58]:
P1 = np.linalg.inv(P)
S1 = np.linalg.inv(S)

In [59]:
P1 

array([[1., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0., 0.]])

In [60]:
c_m = c.dot(P1).astype(int)
c_m

array([ 7, 12, 15, 21,  2, 21])

### Berlekamp-Welch Algorithm

In [61]:
a = np.arange(N, dtype = int)
b = -c_m*a**2 
b = b.astype(int) % n
#b = np.concatenate([[b[0]], [1], b[1:]])

z = np.zeros(N)
z[1] = 1
BW = np.concatenate([[c_m], [c_m*a], [-np.ones(N)], [-a], [-a**2], [-a**3]], axis = 0).T
BW1 =  np.concatenate([[a**4],[a**3],[a**2],[a],[np.ones(N, int)],[-c_m]],axis = 0).T % n
BW = BW.astype(int) % n
BW1

array([[ 0,  0,  0,  0,  1, 16],
       [ 1,  1,  1,  1,  1, 11],
       [16,  8,  4,  2,  1,  8],
       [12,  4,  9,  3,  1,  2],
       [ 3, 18, 16,  4,  1, 21],
       [ 4, 10,  2,  5,  1,  2]])

## Based on prepared code
https://jeremykun.com/2015/09/07/welch-berlekamp/

In [76]:
m

array([1, 2, 3])

In [77]:
enc, dec, solveSystem = makeEncoderDecoder(N, K, 7)
encoded = enc(m)
encoded

[[0 (mod 7), 1 (mod 7)],
 [1 (mod 7), 6 (mod 7)],
 [2 (mod 7), 3 (mod 7)],
 [3 (mod 7), 6 (mod 7)],
 [4 (mod 7), 1 (mod 7)],
 [5 (mod 7), 2 (mod 7)]]

In [78]:
e=1
corrupted = encoded[:]
corrupted[n//2][1] = corrupted[n//2][1] + 1
corrupted

[[0 (mod 7), 1 (mod 7)],
 [1 (mod 7), 6 (mod 7)],
 [2 (mod 7), 3 (mod 7)],
 [3 (mod 7), 0 (mod 7)],
 [4 (mod 7), 1 (mod 7)],
 [5 (mod 7), 2 (mod 7)]]

In [79]:
Q,E = solveSystem(corrupted)
P, remainder = (Q.__divmod__(E))

print("P(x) = %r" % P)
print("r(x) = %r" % remainder)

P(x) = 1 + 2 t^1 + 3 t^2
r(x) = 0
